# Exercice 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

def plot_matrix(matrix, title, mot_pour_labal=None):
    plt.figure(figsize=(8, 6))
    if mot_pour_labal is not None:
        
        import seaborn as sns
        sns.heatmap(matrix, annot=True, cmap='viridis', 
                   xticklabels=mot_pour_labal, yticklabels=mot_pour_labal)
        plt.xlabel('mot_pour_labal')
        plt.ylabel('mot_pour_labal')
    else:
        plt.imshow(matrix, cmap='viridis', aspect='auto')
        plt.colorbar()
        plt.xlabel('Dimension')
        plt.ylabel('Dimension')
    plt.title(title)
    plt.tight_layout()
    plt.show()


input_sequence = np.array([[1, 0, 1, 0, 0.5],   
                           [0, 2, 1, 0, 0.2],
                           [1, 1, 0, 1, 0.8]])

mot_pour_labal = ["mot1", "mot2", "mot3"]
d_model = input_sequence.shape[1] 


print("Input shape:", input_sequence.shape)
print("Input Sequence:\n", input_sequence)

print("Attention une tete ")

query = tf.keras.layers.Dense(d_model)(input_sequence)
key = tf.keras.layers.Dense(d_model)(input_sequence)
value = tf.keras.layers.Dense(d_model)(input_sequence)

plot_matrix(query, "Query Matrix")
plot_matrix(key, "Key Matrix")
plot_matrix(value, "Value Matrix")

scores = tf.matmul(query, key, transpose_b=True)/np.sqrt(d_model)
plot_matrix(scores, "Attention Scores (QK^T)")

attention_weights = tf.nn.softmax(scores, axis=-1)
plot_matrix(attention_weights, "Attention Weights (Softmax)", mot_pour_labal)

attention_output = tf.matmul(attention_weights, value)
plot_matrix(attention_output, "Attention Output")


print("Output Une tete:\n", attention_output.numpy())



print("MULTI-tete ATTENTION ")


def attention_multi_tete(inputs, nb_tete=2):
    d_model = inputs.shape[1]
    d_model_multi = d_model // nb_tete  
    
    ensemble_tete = []
    
    for tete in range(nb_tete):
        print("\n Tete ",tete + 1)
        
        query_tete = tf.keras.layers.Dense(d_model_multi)(inputs)
        key_tete = tf.keras.layers.Dense(d_model_multi)(inputs)
        value_tete = tf.keras.layers.Dense(d_model_multi)(inputs)
        
    
        scores_tete = tf.matmul(query_tete, key_tete, transpose_b=True)/np.sqrt(d_model_multi)
        weights_tete = tf.nn.softmax(scores_tete, axis=-1)
        output_tete = tf.matmul(weights_tete, value_tete)
        
        ensemble_tete.append(output_tete)
        
        print(f"tete {tete + 1} output shape: {output_tete.shape}")
        plot_matrix(weights_tete, f"tete {tete+1} Attention Weightt", mot_pour_labal)
    
    #plusiuers tetes donc chacun sa shape  donc on concatene pour calculer l'output
    concatene = tf.concat(ensemble_tete, axis=-1)
    
    final_output = tf.keras.layers.Dense(d_model)(concatene)
    
    return final_output



multi_output = attention_multi_tete(input_sequence, nb_tete=2)
plot_matrix(multi_output, "Multi-tete Final Output")
print("Multi tete Final Output:\n", multi_output.numpy())


print("Plusieurs echelle")

query_test = tf.keras.layers.Dense(d_model)(input_sequence)
key_test = tf.keras.layers.Dense(d_model)(input_sequence)

scores_normal = tf.matmul(query_test, key_test, transpose_b=True) / np.sqrt(d_model)
weights_normal = tf.nn.softmax(scores_normal)
plot_matrix(weights_normal, 'Scaling sqrt', mot_pour_labal)


scores_fort = tf.matmul(query_test, key_test, transpose_b=True) / 0.5
weights_fort = tf.nn.softmax(scores_fort)
plot_matrix(weights_fort, 'Scaling 0.5 fort', mot_pour_labal)


scores_faible = tf.matmul(query_test, key_test, transpose_b=True) / 5
weights_faible = tf.nn.softmax(scores_faible)
plot_matrix(weights_faible, 'Scaling 5  faible', mot_pour_labal)



print("Analyse !!!")
print("Une tete output shape: ",attention_output.shape)
print("Multi-tete output shape: ",multi_output.shape)
print("Difference moyenne: ",np.mean(np.abs(attention_output.numpy() - multi_output.numpy())))